In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer
from datasets import load_dataset
from tqdm import tqdm
import json
import torch
import argparse
import datasets
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt
from os import listdir
import json
import math
import seaborn as sns

/rds/general/user/jj1122/home/anaconda3/envs/m2d2_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Config

In [2]:
n_layers = 12
list_modules = ['attn', 'mlp']
trace_module_id = "transformer.h.{l}.{m}"

list_trace_module_ids = []

for l in range(n_layers):
    for m in list_modules:
        list_trace_module_ids.append(trace_module_id.format(l=l, m=m))

In [2]:
list_trace_module_ids = ['transformer.h.0.attn.c_attn', 'transformer.h.0.attn.c_proj',
       'transformer.h.0.mlp.c_fc', 'transformer.h.0.mlp.c_proj',
       'transformer.h.1.attn.c_attn', 'transformer.h.1.attn.c_proj',
       'transformer.h.1.mlp.c_fc', 'transformer.h.1.mlp.c_proj',
       'transformer.h.2.attn.c_attn', 'transformer.h.2.attn.c_proj',
       'transformer.h.2.mlp.c_fc', 'transformer.h.2.mlp.c_proj',
       'transformer.h.3.attn.c_attn', 'transformer.h.3.attn.c_proj',
       'transformer.h.3.mlp.c_fc', 'transformer.h.3.mlp.c_proj',
       'transformer.h.4.attn.c_attn', 'transformer.h.4.attn.c_proj',
       'transformer.h.4.mlp.c_fc', 'transformer.h.4.mlp.c_proj',
       'transformer.h.5.attn.c_attn', 'transformer.h.5.attn.c_proj',
       'transformer.h.5.mlp.c_fc', 'transformer.h.5.mlp.c_proj',
       'transformer.h.6.attn.c_attn', 'transformer.h.6.attn.c_proj',
       'transformer.h.6.mlp.c_fc', 'transformer.h.6.mlp.c_proj',
       'transformer.h.7.attn.c_attn', 'transformer.h.7.attn.c_proj',
       'transformer.h.7.mlp.c_fc', 'transformer.h.7.mlp.c_proj',
       'transformer.h.8.attn.c_attn', 'transformer.h.8.attn.c_proj',
       'transformer.h.8.mlp.c_fc', 'transformer.h.8.mlp.c_proj',
       'transformer.h.9.attn.c_attn', 'transformer.h.9.attn.c_proj',
       'transformer.h.9.mlp.c_fc', 'transformer.h.9.mlp.c_proj',
       'transformer.h.10.attn.c_attn', 'transformer.h.10.attn.c_proj',
       'transformer.h.10.mlp.c_fc', 'transformer.h.10.mlp.c_proj',
       'transformer.h.11.attn.c_attn', 'transformer.h.11.attn.c_proj',
       'transformer.h.11.mlp.c_fc', 'transformer.h.11.mlp.c_proj']

In [3]:
gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokeniser = GPT2Tokenizer.from_pretrained("gpt2")

In [4]:
def tokenize_function(examples):
    output = gpt2_tokeniser(examples)
    return output

In [5]:
temp = {}

def func_v1(m_id):
    def hook_v1(module, _input, _output):
        global seq
        dict_sequence[m_id].append(seq)
        seq += 1
        
        module_ = list(module) if isinstance(module, tuple) else module
        input_ = list(_input) if isinstance(_input, tuple) else _input
        output_ = list(_output) if isinstance(_output, tuple) else _output
#         print(type(module_), module_)
        temp[m_id] = [module_, input_, output_]
        
    return hook_v1

gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

for m_id in list_trace_module_ids:
    gpt2.get_submodule(m_id).register_forward_hook(func_v1(m_id))    


In [34]:
input_text = ["hi hello", 'good bye']
data = tokenize_function(input_text)
inputs = torch.tensor(data['input_ids'])
inputs

tensor([[ 5303, 23748],
        [11274, 33847]])

In [35]:
dict_sequence = {x: [] for x in list_trace_module_ids}
seq = 0

with torch.no_grad():
    clean_outputs = gpt2(inputs, labels=inputs.clone())
    clean_loss = np.exp(clean_outputs.loss.item())


## Order

In [36]:
dict_sequence

{'transformer.h.0.attn.c_attn': [0],
 'transformer.h.0.attn.c_proj': [1],
 'transformer.h.0.mlp.c_fc': [2],
 'transformer.h.0.mlp.c_proj': [3],
 'transformer.h.1.attn.c_attn': [4],
 'transformer.h.1.attn.c_proj': [5],
 'transformer.h.1.mlp.c_fc': [6],
 'transformer.h.1.mlp.c_proj': [7],
 'transformer.h.2.attn.c_attn': [8],
 'transformer.h.2.attn.c_proj': [9],
 'transformer.h.2.mlp.c_fc': [10],
 'transformer.h.2.mlp.c_proj': [11],
 'transformer.h.3.attn.c_attn': [12],
 'transformer.h.3.attn.c_proj': [13],
 'transformer.h.3.mlp.c_fc': [14],
 'transformer.h.3.mlp.c_proj': [15],
 'transformer.h.4.attn.c_attn': [16],
 'transformer.h.4.attn.c_proj': [17],
 'transformer.h.4.mlp.c_fc': [18],
 'transformer.h.4.mlp.c_proj': [19],
 'transformer.h.5.attn.c_attn': [20],
 'transformer.h.5.attn.c_proj': [21],
 'transformer.h.5.mlp.c_fc': [22],
 'transformer.h.5.mlp.c_proj': [23],
 'transformer.h.6.attn.c_attn': [24],
 'transformer.h.6.attn.c_proj': [25],
 'transformer.h.6.mlp.c_fc': [26],
 'transform

# ATTN

## attn_c_attn

In [37]:
attn_c_attn = "transformer.h.0.attn.c_attn"

### module

In [38]:
temp[attn_c_attn][0]

Conv1D()

### input

In [39]:
temp[attn_c_attn][1]

[tensor([[[-0.1000, -0.0874, -0.0269,  ..., -0.0154,  0.0138,  0.1415],
          [ 0.1052, -0.1196, -0.0540,  ..., -0.1193,  0.0812,  0.0517]],
 
         [[ 0.0065, -0.0714, -0.0597,  ..., -0.0201, -0.0945, -0.0281],
          [ 0.0837, -0.2269, -0.0595,  ..., -0.3107, -0.1329,  0.0581]]])]

In [40]:
temp[attn_c_attn][1][0].shape

torch.Size([2, 2, 768])

### output (activation, (k, v))

In [41]:
temp[attn_c_attn][2]

tensor([[[ 0.6655, -0.2217,  0.1277,  ...,  0.0662, -0.0129,  0.0362],
         [ 0.4013,  0.8140,  0.4187,  ..., -0.0534, -0.0017,  0.3631]],

        [[-0.0903, -0.4669, -0.4292,  ..., -0.0223, -0.0383,  0.0397],
         [-0.0209,  0.3601, -0.9751,  ...,  0.0634, -0.1967,  0.4776]]])

#### activation $ \in \mathbb{R}^{n\_batch \times n\_tokens \times dim\_model}$

In [42]:
temp[attn_c_attn][2].shape

torch.Size([2, 2, 2304])

## MLP

### module

In [25]:
temp['transformer.h.0.mlp'][0]

GPT2MLP(
  (c_fc): Conv1D()
  (c_proj): Conv1D()
  (act): NewGELUActivation()
  (dropout): Dropout(p=0.1, inplace=False)
)

### input (n_tokens, dim_model)

In [26]:
len(temp['transformer.h.0.mlp'][1])

1

In [27]:
temp['transformer.h.0.mlp'][1][0].shape

torch.Size([2, 2, 768])

### output (n_tokens, dim_model)

In [28]:
len(temp['transformer.h.0.mlp'][2])

2

In [29]:
temp['transformer.h.0.mlp'][2][0].shape

torch.Size([2, 768])

In [30]:
temp['transformer.h.0.mlp'][2][0]

tensor([[-0.1473, -0.8551,  0.2517,  ..., -1.0792,  0.4386, -0.8061],
        [-0.9995, -0.4823, -1.1081,  ..., -0.7381, -0.0529, -1.1930]])